# Brute force solver for a 2x2 Rubik's cube


## Naive 2x2 Brute Force Solver

We use MeepMoops's Py222 implementation:

In [87]:
from py222 import py222

We begin by getting a solved puzzle

In [88]:
s = py222.initState()
print(py222.isSolved(s))
py222.printCube(s)

True
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


Next, we can apply a sequence of moves to scramble the puzzle:

In [103]:
# scramble = py222.doAlgStr(s, "x y R U' R'")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U")
scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U R' U F2")
print(py222.isSolved(scramble))
py222.printCube(scramble)

False
      ┌──┬──┐
      │ 2│ 2│
      ├──┼──┤
      │ 2│ 2│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 0│ 3│ 1│ 4│ 3│ 0│ 4│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 1│ 0│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 5│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘


Next, we brute force a solution.
Note that we can reduce the degrees of freedom to only nine types of moves: 
* Rotations do not affect state, and can be ignored.
* WLOG, moving the left side is equivalent to moving the right side. Thus, instead of 6 turns ((R)ight, (L)eft, (U)p, (D)own, (F)ront, (B)ack), we can use 3 types of turns ((R)ight, (F)ront, and (U)p).
* For each of the three types of turns, we can rotate clockwise ninety degrees, rotate counterclockwise (indicated by a prime (')), or rotate 180 degrees (indicated by a '2')

Note that God's number for a 2x2 Rubik's cube is 11; i.e., any 2x2 puzzle can be solved within 11 moves. This means, using a naive approach, we can bound our run time as follows:
* Our first move has nine possible options
* Subsequent moves have six possible options (WLOG, if we turned the right face on the _n-1_-th move, we won't want to turn right again in any form on the _n_-th move).

Thus, an upper bound for the number of iterations is O(9*6^10) or 544 million iterations for our naive approach.

We can represent the possible moves:

In [90]:
valid_moves = ["R", "R'", "R2", "U", "U'", "U2", "F", "F'", "F2"]

In [91]:
def solve_222(cube):
    cubes_queue = [(cube, [])]

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return [valid_moves[move] for move in prev_moves]

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

        prev_move = prev_moves[-1] if len(prev_moves) > 0 else 9

        # Enforce not repeating the same side again
        next_moves = list(range(9))[0:(prev_move//3 * 3)] + list(range(9))[(prev_move//3 * 3 + 3):]

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_moves:
            next_cube = py222.doAlgStr(cube, valid_moves[move])
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

This appears very slow, so we can try a different method of hashing the cubes we've already tried.

(Unfortunately, this approach is even slower!... likely due to the str(cube) operations.)

In [92]:
def solve_222_bad(cube):
    cubes_already_tried = set([str(cube)])
    cubes_queue = [(cube, [])]

    num_moves = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            return prev_moves

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)))
            num_moves = len(prev_moves)

        # Perform each possible move, and add each of those cubes to the queue
        for move in valid_moves:
            next_cube = py222.doAlgStr(cube, move)
            if str(next_cube) not in cubes_already_tried:
                cubes_queue.append((next_cube, prev_moves + [move]))

In [93]:
solve_222(scramble)

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 3	 Iter: 64
Num moves: 4	 Iter: 388
Num moves: 5	 Iter: 2332
Num moves: 6	 Iter: 13996
Num moves: 7	 Iter: 83980
Num moves: 7	 Iter: 137765


["R'", 'U', 'F2', 'U', 'R', 'U', "R'"]

We now have a naive way to brute force 2x2 solutions. Note that this scales as O(6^n), where n is the number of moves.

## Two-Phase Approach

Next, we try the 2x2 version of Kociemba's algorithm. This involves solving the cube into a "G1" state, where the corners are all oriented. Then, we brute force using a subset of moves: R2, F2, U, U', U2

Recognizing corner orientation is a matter of normalizing* the cube, and then seeing that the top and bottom are all either 0's or 3's.

Normalizing means that we recolor the stickers such that we could have gotten to the state using a single corner we hold fixed.*

In [94]:
def corners_oriented(normalized_cube):
    # This follows the similar idea as the isSolved() method:
    # We check the normalized cube has 0's on the top
    # and 3's on the bottom
    for i in [0, 3]:
        for e in normalized_cube[4 * i:4 * i + 4]:
            if e not in [0, 3]:
                return False
    return True

    a = [0, 1, 2, 3]
    

In [95]:
scr_norm = py222.normFC(scramble)
py222.printCube(scr_norm)
corners_oriented(scr_norm)

      ┌──┬──┐
      │ 1│ 2│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 3│ 1│ 5│ 2│ 1│ 0│ 4│ 2│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 3│ 1│ 2│ 3│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 5│
      ├──┼──┤
      │ 3│ 4│
      └──┴──┘


False

In [96]:
test_corners_oriented = py222.normFC(py222.doAlgStr(s, "x R U' R F2 R' U R"))
py222.printCube(test_corners_oriented)
print(corners_oriented(test_corners_oriented))

      ┌──┬──┐
      │ 0│ 3│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 4│ 2│ 2│ 4│ 1│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 1│ 5│ 4│ 5│ 1│ 2│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 0│
      ├──┼──┤
      │ 3│ 0│
      └──┴──┘
True


Now, we can brute force using 2 steps: we first brute force into a corner oriented state, and then brute force a solution with a more limited move set.

In [97]:
G1_valid_moves = ["R2", "F2", "U", "U'", "U2"]

def next_valid_g1_moves(prev_moves):
    if len(prev_moves) == 0:
        return G1_valid_moves

    prev_move = prev_moves[-1]
    if "U" in prev_move:
        return G1_valid_moves[0:2]
    elif prev_move == "R2":
        return ["F2"] + G1_valid_moves[2:]
    else:
        return ["R2"] + G1_valid_moves[2:]
    
def next_valid_moves(prev_moves):
    prev_move = prev_moves[-1] if len(prev_moves) > 0 else "_"

    # Enforce not repeating the same side again
    return list(filter(lambda move: prev_move[0] not in move, valid_moves))

In [98]:
def solve_to_state(cube, is_in_end_state, next_valid_moves):
    cubes_queue = [(cube, [])]

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if is_in_end_state(cube):
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return [move for move in prev_moves]

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_valid_moves(prev_moves):
            next_cube = py222.doAlgStr(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

def solve_222_multiphase(cube):
    py222.printCube(cube)
    norm_cube = py222.normFC(cube)
    moves_to_g1 = solve_to_state(norm_cube, corners_oriented, next_valid_moves)
    g1_cube = py222.doAlgStr(norm_cube, " ".join(moves_to_g1))
    py222.printCube(g1_cube)
    moves_to_solve = solve_to_state(g1_cube, py222.isSolved, next_valid_g1_moves)
    py222.printCube(py222.doAlgStr(norm_cube, " ".join(moves_to_g1 + moves_to_solve)))
    return moves_to_g1 + moves_to_solve

In [105]:
scramble1 = py222.doAlgStr(s, "x y R U' R'")
scramble2 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R")
scramble3 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U")
scramble4 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U R' U F2")

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 2	 Iter: 28
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 2│ 1│ 5│ 1│ 4│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 4│ 2│ 2│ 1│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 6
Num moves: 3	 Iter: 20
Num moves: 4	 Iter: 64
Num moves: 5	 Iter: 192
Num moves: 6	 Iter: 584
Num moves: 7	 Iter: 1744
Num moves: 8	 Iter: 5256
Num moves: 9	 Iter: 15728
Num moves: 10	 Iter: 47272
Num moves: 11	 Iter: 141648
Num moves: 11	 Iter: 297985
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['U', 'R', 'U', 'R2', "U'", 'R2', "U'", 'F2', 'U', 'F2', "U'", 'F2', 'U']

In [106]:
solve_222_multiphase(scramble1)

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 3	 Iter: 64
      ┌──┬──┐
      │ 0│ 3│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 5│ 1│ 1│ 2│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 5│ 1│ 1│ 2│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 0│
      ├──┼──┤
      │ 3│ 0│
      └──┴──┘
Num moves: 1	 Iter: 1
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R', 'U', 'R', 'R2']

In [107]:
solve_222_multiphase(scramble2)

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 2	 Iter: 28
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 2│ 1│ 5│ 1│ 4│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 4│ 2│ 2│ 1│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 6
Num moves: 3	 Iter: 20
Num moves: 4	 Iter: 64
Num moves: 5	 Iter: 192
Num moves: 6	 Iter: 584
Num moves: 7	 Iter: 1744
Num moves: 8	 Iter: 5256
Num moves: 9	 Iter: 15728
Num moves: 10	 Iter: 47272
Num moves: 11	 Iter: 141648
Num moves: 11	 Iter: 297985
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['U', 'R', 'U', 'R2', "U'", 'R2', "U'", 'F2', 'U', 'F2', "U'", 'F2', 'U']

In [108]:
solve_222_multiphase(scramble3)

Num moves: 1	 Iter: 1
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 2│ 1│ 5│ 1│ 4│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 4│ 2│ 2│ 1│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 6
Num moves: 3	 Iter: 20
Num moves: 4	 Iter: 64
Num moves: 5	 Iter: 192
Num moves: 6	 Iter: 584
Num moves: 7	 Iter: 1744
Num moves: 8	 Iter: 5256
Num moves: 9	 Iter: 15728
Num moves: 10	 Iter: 47272
Num moves: 11	 Iter: 141648
Num moves: 11	 Iter: 297985
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R', 'U', 'R2', "U'", 'R2', "U'", 'F2', 'U', 'F2', "U'", 'F2', 'U']

In [109]:
solve_222_multiphase(scramble4)

Num moves: 0	 Iter: 0
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 1│ 4│ 2│ 5│ 4│ 1│ 5│ 2│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 6
Num moves: 3	 Iter: 20
Num moves: 4	 Iter: 64
Num moves: 5	 Iter: 192
Num moves: 6	 Iter: 584
Num moves: 7	 Iter: 1744
Num moves: 8	 Iter: 5256
Num moves: 9	 Iter: 15728
Num moves: 10	 Iter: 47272
Num moves: 11	 Iter: 141648
Num moves: 12	 Iter: 425288
Num moves: 12	 Iter: 624881
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R2', 'U2', 'F2', 'U', 'R2', 'U2', 'F2', 'U', 'F2', 'U2', 'R2', 'U']